## Módulo de inputs

In [ ]:
import time
import platform
import subprocess
import abc
from enum import Enum, auto
from pynput.keyboard import Controller as KBController, Key

# TECLADO (MOVIMIENTO)

kb = KBController()

KEY_FORWARD = "w"
KEY_LEFT = "a"
KEY_RIGHT = "d"
KEY_JUMP = Key.space

class MoveHold(Enum):
    NEUTRAL = auto()
    FORWARD = auto()
    LEFT = auto()
    RIGHT = auto()

def _release_keys(keys):
    for k in keys:
        try:
            kb.release(k)
        except Exception:
            pass

def _tap(k, press_time=0.03):
    try:
        kb.press(k)
        time.sleep(press_time)
    finally:
        try:
            kb.release(k)
        except Exception:
            pass

class KeyboardMoveManager:
    """Mantiene 1 tecla de movimiento pulsada (W/A/D). Jump es tap independiente."""
    def __init__(self):
        self.current = MoveHold.NEUTRAL

    def set_hold(self, hold: MoveHold):
        if hold == self.current:
            return

        _release_keys([KEY_FORWARD, KEY_LEFT, KEY_RIGHT])

        if hold == MoveHold.FORWARD:
            kb.press(KEY_FORWARD)
        elif hold == MoveHold.LEFT:
            kb.press(KEY_LEFT)
        elif hold == MoveHold.RIGHT:
            kb.press(KEY_RIGHT)

        self.current = hold

    def jump(self):
        _tap(KEY_JUMP, press_time=0.03)

    def release_all(self):
        _release_keys([KEY_FORWARD, KEY_LEFT, KEY_RIGHT])
        self.current = MoveHold.NEUTRAL



# RATÓN (FACTORY)
class MouseInput(abc.ABC):
    """Clase envoltorio para enviar los inputs del ratón al juego"""
    @abc.abstractmethod
    def move(self, dx, dy): pass # Método para enviar movimientos del ratón

    @abc.abstractmethod
    def left_click(self): pass # Método para enviar evento de clic izquierdo


class LinuxMouseInput(MouseInput):
    """Implementación en Linux. Requiere instalar la libreria 'xdotools'."""
    def move(self, dx, dy):
        if dx == 0 and dy == 0:
            return
        try:
            subprocess.Popen(
                ["xdotool", "mousemove_relative", "--", str(int(dx)), str(int(dy))],
                stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL
            )
        except FileNotFoundError:
            print("Error: xdotool no instalado")

    def left_click(self):
        try:
            subprocess.Popen(
                ["xdotool", "click", "1"],
                stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL
            )
        except FileNotFoundError:
            print("Error: xdotool no instalado")


class WindowsMouseInput(MouseInput):
    """Implementación en Windows. Requiere del uso de  'pydirectinput'. Desarrollada por si algún compañero usa Windows"""
    def __init__(self):
        try:
            import pydirectinput
            self.engine = pydirectinput
            self.engine.FAILSAFE = False
            self.available = True
        except ImportError:
            print("Error: pydirectinput no instalado (pip install pydirectinput)")
            self.available = False

    def move(self, dx, dy):
        if not self.available or (dx == 0 and dy == 0):
            return
        self.engine.moveRel(int(dx), int(dy), relative=True)

    def left_click(self):
        if not self.available:
            return
        self.engine.click()


class MouseInputFactory:
    """Factory para crear una instnacia de MouseInput de manera sencilla y sin tener que realizar comprobaciones"""
    @staticmethod
    def create():
        system = platform.system()
        if system == "Linux":
            print("Sistema Linux detectado: Usando xdotool")
            return LinuxMouseInput()
        elif system == "Windows":
            print("Sistema Windows detectado: Usando pydirectinput")
            return WindowsMouseInput()
        else:
            print(f"Sistema {system} no soportado oficialmente")
            return None

## Módulo de detección de gestos.

In [2]:
import time
import math
import os
import urllib.request
from collections import deque
import cv2
import numpy as np
import mediapipe as mp


# CONTROLADOR DE POSE (MEDIAPIPE)
# - Usa MediaPipe para detectar landmarks del cuerpo.
# - Hace una calibración (neutral).
# - En ejecución decide: izquierda/derecha, avanzar (forward) y salto (jump).

LS, RS = 11, 12
LH, RH = 23, 24
MIN_VIS = 0.60

# Tiempos de calibración (espera inicial, neutral, forward)
PRE_CALIB_WAIT_SECONDS = 5.0
CALIB_NEUTRAL_SECONDS = 2.0
CALIB_FORWARD_SECONDS = 1.2

# Umbrales y filtros para detectar inclinación lateral y forward (con histéresis y suavizado)
LEAN_X_THRESH = 0.25
EMA_DY = 0.35
DY_ENTER_MIN = 0.030
DY_STD_MULT = 6.0
DY_FWD_FRAC = 0.70
DY_EXIT_RATIO = 0.65
STABLE_FRAMES_FWD = 3
DY_UP_BLOCK_MIN = 0.020

# Umbrales y filtros para detectar salto (altura + “velocidad”) y evitar dobles disparos
EMA_Y = 0.30
JUMP_UP_HIP = 0.10
JUMP_VEL = 0.06
JUMP_COOLDOWN_SEC = 0.55
JUMP_PREBLOCK_UP = 0.06
JUMP_PREBLOCK_VEL = 0.03
STABLE_FRAMES_SIDE = 2

# Conexiones entre landmarks para dibujar el esqueleto en pantalla (debug/visualización)
POSE_CONNECTIONS = [
    (11, 12), (11, 23), (12, 24), (23, 24),
    (11, 13), (13, 15), (12, 14), (14, 16),
    (23, 25), (25, 27), (24, 26), (26, 28),
    (27, 31), (28, 32), (27, 29), (28, 30),
    (29, 31), (30, 32)
]

def _vis(lm):
    """Lee la visibilidad del landmark (si no existe, asume 1.0)"""
    return float(getattr(lm, "visibility", 1.0))

def draw_pose(frame_bgr, lms):
    """ Dibuja puntos y líneas de la pose sobre el frame (útil para ver qué está detectando)"""
    h, w = frame_bgr.shape[:2]
    for lm in lms:
        cv2.circle(frame_bgr, (int(lm.x*w), int(lm.y*h)), 3, (255,255,255), -1)
    for a, b in POSE_CONNECTIONS:
        ax, ay = int(lms[a].x*w), int(lms[a].y*h)
        bx, by = int(lms[b].x*w), int(lms[b].y*h)
        cv2.line(frame_bgr, (ax, ay), (bx, by), (255,255,255), 2)

def center_norm(lms, a, b):
    """ Punto medio entre dos landmarks (sirve para centros de hombros y caderas)"""
    return (
        (lms[a].x + lms[b].x) * 0.5,
        (lms[a].y + lms[b].y) * 0.5,
        (lms[a].z + lms[b].z) * 0.5
    )

def extract_features(lms_norm):
    """Extrae las señales mínimas para controlar:
     - lean_x: inclinación lateral normalizada (izq/der)
     - shy: altura del centro de hombros (para forward)
     - hip_y_norm: altura de cadera normalizada (para salto)"""
    if not all(_vis(lms_norm[i]) >= MIN_VIS for i in (LS, RS, LH, RH)):
        return None

    shx, shy, _ = center_norm(lms_norm, LS, RS)
    hix, hiy, _ = center_norm(lms_norm, LH, RH)

    shoulder_w = abs(lms_norm[LS].x - lms_norm[RS].x) + 1e-6
    lean_x = (shx - hix) / shoulder_w

    torso_len = abs(hiy - shy) + 1e-6
    hip_y_norm = hiy / torso_len

    return {"lean_x": lean_x, "shy": shy, "hip_y_norm": hip_y_norm}

class Debouncer:
    """ Antirrebote: evita que la acción cambie por ruido exigiendo N frames seguidos iguales"""
    def __init__(self):
        self.last = None
        self.count = 0
    def reset(self):
        self.last = None
        self.count = 0
    def update(self, proposed, need):
        if proposed == self.last:
            self.count += 1
        else:
            self.last = proposed
            self.count = 1
        return self.count >= need

class PoseMovementController:
    """ Controlador principal:
     - Descarga el modelo si hace falta
     - Inicializa PoseLandmarker en modo VIDEO (requiere timestamp)
     - Calibra neutral/forward
     - En RUN decide movimientos a partir de las features"""
    MODEL_URL = "https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_lite/float16/1/pose_landmarker_lite.task"
    MODEL_PATH = "pose_landmarker_lite.task"

    def __init__(self):
        self._ensure_model()
        self._init_landmarker()

        # Base de reloj monótono para generar timestamps consistentes
        self._clock0 = time.monotonic()
        self._last_ts_ms = 0

        self.reset()

    def _ensure_model(self):
        """ Si no existe el archivo del modelo, lo descarga"""
        if not os.path.exists(self.MODEL_PATH):
            print("Descargando modelo PoseLandmarker...")
            urllib.request.urlretrieve(self.MODEL_URL, self.MODEL_PATH)
            print("OK:", self.MODEL_PATH)

    def _init_landmarker(self):
        """ Crea el detector de pose con umbrales de confianza y modo VIDEO"""
        BaseOptions = mp.tasks.BaseOptions
        PoseLandmarker = mp.tasks.vision.PoseLandmarker
        PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
        RunningMode = mp.tasks.vision.RunningMode

        self._landmarker = PoseLandmarker.create_from_options(
            PoseLandmarkerOptions(
                base_options=BaseOptions(model_asset_path=self.MODEL_PATH),
                running_mode=RunningMode.VIDEO,
                num_poses=1,
                min_pose_detection_confidence=0.5,
                min_pose_presence_confidence=0.5,
                min_tracking_confidence=0.5
            )
        )

    def reset(self):
        """ Reinicia el flujo de calibración y los estados internos (filtros, historial, debouncer)"""
        self.stage = "WAIT"
        self.wait_start = None
        self.calib_start = None
        self.neu_feats = []
        self.fwd_feats = []
        self.neutral = None

        self.dy_s = 0.0
        self.y_s = 0.0
        self.y_hist = deque(maxlen=10)
        self.last_jump_time = 0.0
        self.deb = Debouncer()
        self.current_hold = MoveHold.NEUTRAL  # viene de la CELDA 1

    def close(self):
        """ Cierre seguro del landmarker"""
        try:
            self._landmarker.close()
        except Exception:
            pass

    def _decide(self, feat):
        """ Lógica de decisión:
         1) Prioriza lateral (izq/der) si la inclinación supera el umbral.
         2) Detecta salto con altura + velocidad + cooldown.
         3) Detecta forward con umbral dinámico + histéresis y bloqueos para no mezclarlo con salto."""
        lean_dx = feat["lean_x"] - self.neutral["lean0"]
        if abs(lean_dx) > LEAN_X_THRESH:
            action = MoveHold.RIGHT if lean_dx > 0 else MoveHold.LEFT
            return action, False, None

        y_raw = feat["hip_y_norm"] - self.neutral["hip_y0"]
        self.y_s = EMA_Y * y_raw + (1.0 - EMA_Y) * self.y_s
        self.y_hist.append(y_raw)

        vel = 0.0
        if len(self.y_hist) >= 4:
            vel = (self.y_hist[-4] - self.y_hist[-1])

        up = -(self.y_s)
        now = time.time()
        can_jump = (now - self.last_jump_time) >= JUMP_COOLDOWN_SEC
        do_jump = can_jump and (up > JUMP_UP_HIP) and (vel > JUMP_VEL)
        preblock_jump = (up > JUMP_PREBLOCK_UP and vel > JUMP_PREBLOCK_VEL)

        dy_raw = (feat["shy"] - self.neutral["shy0"])
        self.dy_s = EMA_DY * dy_raw + (1.0 - EMA_DY) * self.dy_s

        dy_enter = max(DY_ENTER_MIN, DY_STD_MULT * self.neutral["dy_std"], DY_FWD_FRAC * self.neutral["dy_fwd_delta"])
        dy_exit = dy_enter * DY_EXIT_RATIO

        if self.dy_s < -DY_UP_BLOCK_MIN:
            fb = MoveHold.NEUTRAL
        else:
            if self.current_hold != MoveHold.FORWARD and (do_jump or preblock_jump):
                fb = MoveHold.NEUTRAL
            else:
                if self.current_hold == MoveHold.FORWARD:
                    fb = MoveHold.FORWARD if self.dy_s > dy_exit else MoveHold.NEUTRAL
                else:
                    fb = MoveHold.FORWARD if self.dy_s > dy_enter else MoveHold.NEUTRAL

        return fb, do_jump, None

    def process_frame(self, frame_bgr):
        """ Procesamiento de frame:
         - Convierte a RGB y crea mp.Image
         - Genera timestamp monótono creciente (requisito del modo VIDEO)
         - Detecta pose, dibuja y extrae features
         - Si no está calibrado: avanza por etapas WAIT/NEUTRAL/FORWARD y calcula baseline
         - Si está calibrado: decide acción con debouncer y gestiona cooldown de salto"""
        rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb)

        timestamp_ms = int((time.monotonic() - self._clock0) * 1000)
        if timestamp_ms <= self._last_ts_ms:
            timestamp_ms = self._last_ts_ms + 1
        self._last_ts_ms = timestamp_ms

        result = self._landmarker.detect_for_video(mp_image, timestamp_ms)
        feat = None

        if result.pose_landmarks:
            lms = result.pose_landmarks[0]
            draw_pose(frame_bgr, lms)
            feat = extract_features(lms)

        do_jump = False

        if self.stage != "DONE":
            self.current_hold = MoveHold.NEUTRAL

            if self.stage == "WAIT":
                if feat is None:
                    self.wait_start = None
                else:
                    if self.wait_start is None:
                        self.wait_start = time.time()
                    if (time.time() - self.wait_start) >= PRE_CALIB_WAIT_SECONDS:
                        self.stage = "NEUTRAL"
                        self.calib_start = None
                        self.neu_feats.clear()
                        self.fwd_feats.clear()

            elif self.stage in ("NEUTRAL", "FORWARD"):
                if feat is None:
                    pass
                else:
                    if self.calib_start is None:
                        self.calib_start = time.time()
                        if self.stage == "NEUTRAL":
                            self.neu_feats.clear()
                        else:
                            self.fwd_feats.clear()

                    elapsed = time.time() - self.calib_start

                    if self.stage == "NEUTRAL":
                        self.neu_feats.append(feat)
                        if elapsed >= CALIB_NEUTRAL_SECONDS and len(self.neu_feats) >= 10:
                            self.stage = "FORWARD"
                            self.calib_start = None

                    elif self.stage == "FORWARD":
                        self.fwd_feats.append(feat)
                        if elapsed >= CALIB_FORWARD_SECONDS and len(self.fwd_feats) >= 8:
                            lean0 = sum(f["lean_x"] for f in self.neu_feats) / len(self.neu_feats)
                            shy0 = sum(f["shy"] for f in self.neu_feats) / len(self.neu_feats)
                            hip_y0 = sum(f["hip_y_norm"] for f in self.neu_feats) / len(self.neu_feats)

                            dys = [f["shy"] for f in self.neu_feats]
                            dy_std = math.sqrt(sum((x - shy0) ** 2 for x in dys) / len(dys))

                            shy_fwd = sum(f["shy"] for f in self.fwd_feats) / len(self.fwd_feats)
                            dy_fwd_delta = max(0.0, shy_fwd - shy0)

                            self.neutral = {
                                "lean0": lean0,
                                "shy0": shy0,
                                "dy_std": dy_std,
                                "dy_fwd_delta": dy_fwd_delta,
                                "hip_y0": hip_y0
                            }
                            self.stage = "DONE"
                            self.deb.reset()
                            self.dy_s = 0.0
                            self.y_s = 0.0
                            self.y_hist.clear()
                            self.current_hold = MoveHold.NEUTRAL

        else:
            if feat is None:
                self.current_hold = MoveHold.NEUTRAL
                do_jump = False
                self.dy_s = 0.0
                self.y_s = 0.0
                self.y_hist.clear()
                self.deb.reset()
            else:
                proposed_raw, do_jump, _ = self._decide(feat)

                need = STABLE_FRAMES_FWD if proposed_raw == MoveHold.FORWARD else STABLE_FRAMES_SIDE
                if self.deb.update(proposed_raw, need):
                    self.current_hold = proposed_raw

                if do_jump:
                    self.last_jump_time = time.time()

        return self.current_hold, do_jump, self.stage, None

## Módulo de seguimiento del objeto apuntador.

In [ ]:
import time
import platform
import subprocess
import cv2
import numpy as np

# pygetwindow (solo Windows)
if platform.system() == "Windows":
    try:
        import pygetwindow as gw
    except Exception:
        gw = None
else:
    gw = None


# SEGUIDOR DE LA PISTOLA
# Tracker del sistema. Utilizar el principalmente el algoritmo CSRT con un filtro de Kalman.
# Calibra al principio mediante 3 perfiles (9 clicks). Si se pierde el algoritmo CSRT busca con
# los perfiles calibrados en un área creciente alrededor de la zona que predice el filtro de Kalman. 
# Si no lo encuentra pasado un cierto número de tiempo, busca en toda la imagen.
class ObjectTracker:
    def __init__(self):
        """
        Inicializa la cámara, las variables de estado y la conexión con el ratón.
        Configura los umbrales de brillo para el disparo y la sensibilidad del movimiento.
        """
        self.cs2_active = False # Indica si la ventana del CS2 está activa y en primer plano
        self.camera_control_enabled = False
        self.sensitivity = 0.5 # Sensibilidades eje X e Y
        self.sensitivity_y = 0.5
        self.COUNTER_STRIKE_WINDOW_NAME = "Counter-Strike 2"

        self.mouse_input = MouseInputFactory.create()

        self.shoot_detection_enabled = True
        self.brightness_threshold = 25 # Umbral para detectar el disparo de la pistola usada en el proyecto
        self.last_shoot_time = 0
        self.shoot_cooldown = 0.1
        self.reset_state()

    def reset_state(self):
        """
        Reinicia todos los contadores, listas de texturas y estados del tracker.
        Se llama al iniciar o al pulsar 'R' para recalibrar desde cero.
        """
        self.texture_maps = [] # Lista de recortes tomados de la pistola por la cámara
        self.target_colors_hsv = [] # Lista de naranjas extraidos de la pistola por la cámara
        self.all_clicks_history = [] # Historial de los clicks de las calibraciones
        self.calibration_stage = 0 # Fase del tracker (0, 1, 2 y 3)
        self.current_clicks_coords = [] # Lista de los clics actuales
        self.roi = None # Roi del objeto trackeado
        self.lost_frames = 0
        self.max_lost_frames = 30
        self.search_expansion = False # Indica si está en modo búsqueda expandida (va creciendo alrededor de la predicción del filtro de Kalman)
        self.pending_click = None # Clic a procesar
        self.tracker = None # Se encarga de tener la referencia a la instancia del algoritmo CSRT 
        self.show_roi = True
        self.texture_match_threshold = 0.70 # Umbral para tener en cuenta con la semejanza de la textura
        self.search_window = 100 # Tamaño ventana de búsqueda
        if hasattr(self, "ambient_brightness"):
            delattr(self, "ambient_brightness")

    def get_active_window_title(self):
        """
        Obtiene el título de la ventana que está en primer plano en el sistema operativo.
        """
        system_name = platform.system()
        try:
            if system_name == "Windows" and gw is not None:
                window = gw.getActiveWindow()
                return window.title if window else ""
            else:
                result = subprocess.check_output(
                    ["xdotool", "getactivewindow", "getwindowname"],
                    stderr=subprocess.STDOUT
                )
                return result.decode("utf-8").strip()
        except Exception:
            return ""

    def adjust_mouse_sensitivity(self, delta, min_val=0.0, max_val=5.0):
        """Cambia la sensibilidad del movimiento enviado al juego."""
        self.sensitivity = max(min_val, min(max_val, self.sensitivity + delta))
        self.sensitivity_y = max(min_val, min(max_val, self.sensitivity_y + delta))

    def camera_tracking_to_mouse_input(self, cx, cy, frame_shape):
        """
        Calcula el desplazamiento necesario desde el centro de la pantalla hasta el objeto
        y envía el input al juego con la sensibilidad configurada.
        """
        if not self.camera_control_enabled or not self.cs2_active or not self.mouse_input:
            return
        h, w = frame_shape[:2]
        move_x = int((cx - w // 2) * self.sensitivity)
        move_y = int((cy - h // 2) * self.sensitivity_y)
        self.mouse_input.move(move_x, move_y)

    def detect_shoot(self, frame):
        """
        Detecta cambios bruscos de brillo (fogonazos) en la zona de seguimiento (ROI)
        para enviar el comando de disparo al juego.
        - Convierte la ROI a escala de grises.
        - Calcula la media aritmética (np.mean) que es muy rápida.
        - Compara con el brillo ambiente.
        - Si supera el umbral, dispara.
        """
        if (not self.shoot_detection_enabled or
            not self.camera_control_enabled or
            not self.cs2_active or
            self.roi is None):
            return False

        # Cooldown de disparo.
        current_time = time.time()
        if current_time - self.last_shoot_time < self.shoot_cooldown:
            return False

        x, y, w, h = [int(v) for v in self.roi] # Dimensiones de la roi como enteros
        fh, fw = frame.shape[:2] # Alto y Ancho de la pantalla
        x1, y1, x2, y2 = max(0, x), max(0, y), min(fw, x + w), min(fh, y + h)

        if x2 <= x1 or y2 <= y1:
            return False

        roi_bgr = frame[y1:y2, x1:x2]
        gray_roi = cv2.cvtColor(roi_bgr, cv2.COLOR_BGR2GRAY)
        current_intensity = np.mean(gray_roi)

        if not hasattr(self, "ambient_brightness"):
            self.ambient_brightness = current_intensity

        diff = current_intensity - self.ambient_brightness
        if diff > self.brightness_threshold:
            self.last_shoot_time = current_time
            if self.mouse_input:
                self.mouse_input.left_click()
            return True
        else:
            self.ambient_brightness = self.ambient_brightness * 0.8 + current_intensity * 0.2 # Adaptación a la luz ambiente
            return False

    def mouse_callback(self, event, x, y, flags, param):
        """Captura los clics del usuario durante la fase de calibración."""
        if event == cv2.EVENT_LBUTTONDOWN:
            self.pending_click = (x, y)

    def process_pending_click(self, frame):
        """Procesa el clic guardado, almacenando la coordenada para definir el perfil del objeto."""
        if self.pending_click is None:
            return
        x, y = self.pending_click
        self.pending_click = None

        self.current_clicks_coords.append((x, y))
        self.all_clicks_history.append((x, y))
        print(f"Click {len(self.current_clicks_coords)}/3 guardado.")

        if len(self.current_clicks_coords) >= 3:
            self.finalize_stage(frame)

    def finalize_stage(self, frame):
        """
        Se ejecuta tras la calibración de 3 fases (9 clics).
        - Calcula el círculo que encierra los puntos.
        - Obtiene el cuadrado que tiene esa circunferencia circunscrita
        - Recorta esa zona como 'Textura Maestra'.
        - Calcula el color HSV promedio de esa zona.
        - Guarda ambos datos y avanza a la siguiente fase de la calibración.
        """
        points = np.array(self.current_clicks_coords, dtype=np.int32)
        (cx, cy), radius = cv2.minEnclosingCircle(points) # Se queda con la circunferencia más pequeña capaz de obtener esos puntos

        # Con la circunferencia obtiene el cuadrado que la tiene circunscrita
        padding = 5
        side = int(radius * 2) + padding
        rx = int(cx - radius - padding/2)
        ry = int(cy - radius - padding/2)
        h_frame, w_frame = frame.shape[:2]

        rx = max(0, rx)
        ry = max(0, ry)
        rw = min(side, w_frame - rx)
        rh = min(side, h_frame - ry)

        if rw > 0 and rh > 0: # Si el recorte existe
            texture_map = frame[ry:ry+rh, rx:rx+rw].copy()
            self.texture_maps.append(texture_map) # Se guarda una copia de la imagen en ese cuadrado
            hsv_roi = cv2.cvtColor(texture_map, cv2.COLOR_BGR2HSV)
            avg_hsv = np.mean(hsv_roi, axis=(0, 1))
            self.target_colors_hsv.append(avg_hsv) # Se guarda la media del color en hsv en el roi
            print(f"Mapa de Texturas {self.calibration_stage + 1} generado.")

            self.current_clicks_coords = []
            self.calibration_stage += 1

            if self.calibration_stage == 3:
                self.roi = (rx, ry, rw, rh)
                self.initialize_tracker_auto(frame) # Se manda a inicializar el tracker

    def initialize_tracker_auto(self, frame):
        """Inicializa el tracker CSRT y el filtro Kalman con la última ROI calculada."""
        if not hasattr(cv2, "TrackerCSRT_create"):
            raise RuntimeError("Falta CSRT: instala opencv-contrib-python")
        x, y, w, h = self.roi
        cx, cy = x + w//2, y + h//2
        self.tracker = cv2.TrackerCSRT_create()
        self.tracker.init(frame, self.roi)
        self.init_kalman(cx, cy)
        print("TRACKER INICIADO.")

    def detect_by_texture(self, frame, pred_x, pred_y, full_scan=False):
        """
        Motor de búsqueda visual con estrategia de 'Tolerancia Cero' para eliminar falsos positivos.
        Combina Template Matching (Forma) con un filtrado estricto de Color (HSV).

        Estrategia de Filtrado:
        - Zona de Búsqueda:
           - Normal: Busca solo alrededor de la predicción de Kalman
           - Pánico: Busca en toda la imagen si el objeto se ha perdido > 0.5s.

        - Escalas:
           - Prueba variaciones de tamaño para tolerar que el usuario se aleje/acerque.

        - Vetos de Color:
           - HUE: Margen estricto de 15 grados. Si no es el naranja exacto, se descarta (intentando evitar que siga otras cosas)
           - SAT: Mínimo 60. Si el candidato es grisáceo o sombra, se descarta.

        - Umbrales de Confianza:
           - Modo Local: Exige > 0.70 de coincidencia.
           - Modo Pánico: Exige > 0.80 para intentar evitar reenganchar con otras cosas en la imagen.
        """
        if not self.texture_maps:
            return None
        h_frame, w_frame = frame.shape[:2]

        # Definir Zona de Búsqueda
        if full_scan:
            sx1, sy1 = 0, 0
            search_area = frame
        else:
            sw = self.search_window
            if self.search_expansion:
                sw = int(sw * 1.5)
            sx1 = max(0, pred_x - sw)
            sy1 = max(0, pred_y - sw)
            sx2 = min(w_frame, pred_x + sw)
            sy2 = min(h_frame, pred_y + sw)
            search_area = frame[sy1:sy2, sx1:sx2]

        if search_area.size == 0:
            return None

        best_score = -1
        best_rect = None

        # En Full Scan reducimos escalas para rendimiento; en local probamos zoom
        scales = [1.0, 0.9, 0.8, 0.7, 0.6, 0.5] if not full_scan else [1.0]

        for i, texture_map in enumerate(self.texture_maps):
            target_color = self.target_colors_hsv[i]
            for scale in scales:
                # Redimensionado del Template (Zoom)
                if scale != 1.0:
                    t_w = int(texture_map.shape[1] * scale)
                    t_h = int(texture_map.shape[0] * scale)
                    if t_w < 10 or t_h < 10:
                        continue
                    try:
                        curr_template = cv2.resize(texture_map, (t_w, t_h))
                    except Exception:
                        continue
                else:
                    curr_template = texture_map

                # Chequeo de límites
                if (curr_template.shape[0] >= search_area.shape[0] or
                    curr_template.shape[1] >= search_area.shape[1]):
                    continue

                try:
                    # Encaje
                    res = cv2.matchTemplate(search_area, curr_template, cv2.TM_CCOEFF_NORMED)
                    _, max_v, _, max_loc = cv2.minMaxLoc(res)

                    # Filtro preliminar
                    if max_v < self.texture_match_threshold:
                        continue

                    lx, ly = max_loc
                    cand_w, cand_h = curr_template.shape[1], curr_template.shape[0]
                    candidate_roi = search_area[ly:ly+cand_h, lx:lx+cand_w]

                    final_score = max_v
                    if candidate_roi.size > 0:
                        # Filtros de color
                        cand_hsv = cv2.cvtColor(candidate_roi, cv2.COLOR_BGR2HSV)
                        cand_avg = np.mean(cand_hsv, axis=(0, 1))
                        diff_h = abs(cand_avg[0] - target_color[0])
                        if diff_h > 90:
                            diff_h = 180 - diff_h

                        # Si varía más de 15 grados, es otro color -> DESCARTAR
                        if diff_h > 15:
                            continue
                        # Veto de Saturación (Anti-Sombras/Grises)
                        if cand_avg[1] < 60:
                            continue

                        diff_s = abs(cand_avg[1] - target_color[1])
                        penalty = (diff_h * 0.005) + (diff_s * 0.002)
                        final_score = max_v - penalty

                    # Umbral Final
                    # Si estamos perdidos, exigimos 80% de seguridad.
                    # Si estamos trackeando, exigimos 65%.
                    required_score = 0.80 if full_scan else self.texture_match_threshold
                    if final_score > required_score and final_score > best_score:
                        best_score = final_score
                        global_x = sx1 + lx
                        global_y = sy1 + ly
                        cx_new = global_x + cand_w // 2
                        cy_new = global_y + cand_h // 2
                        best_rect = (cx_new, cy_new, cand_w, cand_h)

                except Exception:
                    continue

        # Filtro de Distancia Euclidiana
        # Si el Kalman dice que está en X, y encontramos algo muy lejos, lo descartamos.
        if best_rect and not full_scan:
            bx, by = best_rect[0], best_rect[1]
            dist = np.sqrt((bx - pred_x)**2 + (by - pred_y)**2)
            max_dist = 150 if self.search_expansion else 100
            if dist > max_dist:
                return None

        return best_rect

    def detect_object(self, frame):
        """
        Orquestador de la detección del objeto a seguir.

        Estrategia Escalonada:
        - Intenta tracking rápido (CSRT).
        - Si falla, usa Kalman para predecir.
        - Intenta recuperación por textura buscando en el área predicha por el filtro de Kalman.
        - Si lleva > 30 frames perdido, intenta recuperación por textura denominado modo pánico.
        """
        if self.calibration_stage < 3:
            return None, None

        success, bbox = self.tracker.update(frame)
        if success:
            # Tracking CSRT Exitoso
            x, y, w, h = [int(v) for v in bbox]
            cx, cy = x + w // 2, y + h // 2
            self.kalman.correct(np.array([[np.float32(cx)], [np.float32(cy)]]))
            self.roi = (x, y, w, h)
            self.lost_frames = 0
            self.search_expansion = False
            self.camera_tracking_to_mouse_input(cx, cy, frame.shape)

            mask = np.zeros(frame.shape[:2], dtype=np.uint8)
            cv2.rectangle(mask, (x, y), (x+w, y+h), 255, -1)
            return (cx, cy), mask

        # Tracking Fallido -> Usar Predicción + Recuperación
        prediction = self.kalman.predict()
        pred_cx, pred_cy = int(prediction[0]), int(prediction[1])

        self.lost_frames += 1
        force_full_scan = self.lost_frames > 30 # Activar escaneo global si llevamos tiempo perdidos

        result = self.detect_by_texture(frame, pred_cx, pred_cy, full_scan=force_full_scan)
        if result:
            # Si recupera algo
            cx, cy, w_new, h_new = result
            self.kalman.correct(np.array([[np.float32(cx)], [np.float32(cy)]]))
            # Resetear Kalman para la nueva posición
            self.kalman.statePost = np.array([[np.float32(cx)], [np.float32(cy)], [0], [0]], np.float32)

            x_roi = int(cx - w_new/2)
            y_roi = int(cy - h_new/2)
            self.roi = (x_roi, y_roi, w_new, h_new)

            self.tracker = cv2.TrackerCSRT_create()
            self.tracker.init(frame, self.roi)
            self.lost_frames = 0
            self.search_expansion = False
            self.camera_tracking_to_mouse_input(cx, cy, frame.shape)
            return (cx, cy), None

        self.search_expansion = True
        if force_full_scan:
            return (None, None)
        return ((pred_cx, pred_cy), None) if self.lost_frames < 20 else (None, None)

    def init_kalman(self, x, y):
        """
        Configura el Filtro de Kalman.
        Modelo de velocidad constante (posición x,y y velocidad vx, vy).
        Ajustado para movimientos rápidos de ratón (baja inercia).
        """
        self.kalman = cv2.KalmanFilter(4, 2)
        self.kalman.measurementMatrix = np.array([[1,0,0,0], [0,1,0,0]], np.float32)
        self.kalman.transitionMatrix = np.array([[1,0,1,0], [0,1,0,1], [0,0,1,0], [0,0,0,1]], np.float32)
        self.kalman.processNoiseCov = np.eye(4, dtype=np.float32) * 1.0
        self.kalman.statePost = np.array([[np.float32(x)], [np.float32(y)], [0], [0]], np.float32)
        self.kalman.statePre = self.kalman.statePost.copy()

## Módulo De ejecución principal

In [ ]:
import cv2

# Si lo pones True, el movimiento SOLO se enviará si detecta CS2 en primer plano.
RESTRICT_MOVEMENT_TO_CS2 = False

def _stage_color(stage: int):
    if stage == 0:
        return (0, 255, 255)   # amarillo
    if stage == 1:
        return (255, 100, 0)   # naranja
    if stage == 2:
        return (0, 100, 255)   # azul
    return (0, 255, 0)

def _draw_click_points(display, tracker: ObjectTracker):
    """
    Puntos GRANDES y visibles donde pinchas:
    - Historial (hasta 9 clicks) con color por fase y número 1..3.
    - Puntos actuales de la fase con contorno blanco.
    """
    for idx, (px, py) in enumerate(tracker.all_clicks_history):
        stage = min(idx // 3, 2)
        num = (idx % 3) + 1
        col = _stage_color(stage)
        cv2.circle(display, (px, py), 4, (0, 0, 0), 2)
        cv2.putText(display, str(num), (px + 10, py - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, col, 2)

    if tracker.calibration_stage < 3:
        col = _stage_color(tracker.calibration_stage)
        for i, (px, py) in enumerate(tracker.current_clicks_coords):
            cv2.circle(display, (px, py), 4, (255, 255, 255), 2)
            cv2.putText(display, str(i + 1), (px + 12, py + 14),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

def main():
    global RESTRICT_MOVEMENT_TO_CS2
    WIN = "CS2-Control (Pose + Tracker)"
    cv2.namedWindow(WIN, cv2.WINDOW_NORMAL)

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise RuntimeError("No pude abrir la cámara (VideoCapture(0)).")

    pose_ctrl = PoseMovementController()
    kb_mgr = KeyboardMoveManager()
    tracker = ObjectTracker()

    movement_control_enabled = False
    frame_count = 0

    def on_mouse(event, x, y, flags, param):
        tracker.mouse_callback(event, x, y, flags, param)
    cv2.setMouseCallback(WIN, on_mouse)

    try:
        while True:
            ok, frame = cap.read()
            if not ok:
                break

            frame = cv2.flip(frame, 1)
            display = frame.copy()
            h, w = display.shape[:2]

            # Detectar Ventana Activa
            frame_count += 1
            if frame_count % 30 == 0:
                title = tracker.get_active_window_title()
                tracker.cs2_active = (tracker.COUNTER_STRIKE_WINDOW_NAME in title)


            # LOGICA TRACKER (PISTOLA)

            if tracker.calibration_stage < 3:
                tracker.process_pending_click(frame)

            shot = False
            result_pos = (None, None)

            if tracker.calibration_stage == 3:
                shot = tracker.detect_shoot(frame)
                result_pos, _mask = tracker.detect_object(frame)

            # DIBUJAR TRACKER HUD
            cv2.line(display, (w // 2, 0), (w // 2, h), (0, 255, 255), 1)
            cv2.line(display, (0, h // 2), (w, h // 2), (0, 255, 255), 1)

            if tracker.calibration_stage == 0:
                txt, col = f"1. CALIBRA PISTOLA: DERECHA ({len(tracker.current_clicks_coords)}/3)", _stage_color(0)
            elif tracker.calibration_stage == 1:
                txt, col = f"2. CALIBRA PISTOLA: IZQUIERDA ({len(tracker.current_clicks_coords)}/3)", _stage_color(1)
            elif tracker.calibration_stage == 2:
                txt, col = f"3. CALIBRA PISTOLA: CENTRO ({len(tracker.current_clicks_coords)}/3)", _stage_color(2)
            else:
                if tracker.camera_control_enabled and tracker.cs2_active:
                    txt = "JUEGO ACTIVO: PISTOLA + MOV"
                    col = (0, 255, 0)
                else:
                    txt, col = "PISTOLA OK. (Pulsa C para activar raton)", (0, 255, 0)

            cv2.putText(display, txt, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, col, 2)

            # Puntos visibles donde pinchas
            _draw_click_points(display, tracker)

            # DIBUJAR ROI TRACKER
            if tracker.calibration_stage == 3 and tracker.roi and tracker.show_roi:
                rx, ry, rw, rh = tracker.roi
                col_roi = (0,0,255) if tracker.lost_frames > tracker.max_lost_frames else (0,255,0)
                cv2.rectangle(display, (rx, ry), (rx+rw, ry+rh), col_roi, 2)

            if result_pos and result_pos[0] is not None:
                cx, cy = result_pos
                cv2.circle(display, (cx, cy), 8, (0, 255, 255), -1)


            # 2. LOGICA POSE (CUERPO) -> GATED por Tracker

            pose_stage_display = "ESPERANDO PISTOLA..."
            hold = MoveHold.NEUTRAL
            do_jump = False

            if tracker.calibration_stage >= 3:
                hold, do_jump, stage, _ = pose_ctrl.process_frame(display)

                if stage == "WAIT":
                    pose_stage_display = "4. POSE: ENTRA EN CAMARA..."
                elif stage == "NEUTRAL":
                    pose_stage_display = "5. POSE: QUIETO (NEUTRAL)..."
                elif stage == "FORWARD":
                    pose_stage_display = "6. POSE: INCLINATE (FORWARD)..."
                elif stage == "DONE":
                    pose_stage_display = f"ACCION: {hold.name}" + ("+JUMP" if do_jump else "")

                    if not movement_control_enabled:
                        movement_control_enabled = True
                        print(">> MOVIMIENTO ACTIVADO AUTOMATICAMENTE <<")

            cv2.putText(display, pose_stage_display, (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)


            # 3. APLICAR INPUTS (MOVIMIENTO)

            allow_movement = (
                movement_control_enabled and
                tracker.calibration_stage >= 3 and
                pose_ctrl.stage == "DONE" and
                (not RESTRICT_MOVEMENT_TO_CS2 or tracker.cs2_active)
            )

            if allow_movement:
                kb_mgr.set_hold(hold)
                if do_jump:
                    kb_mgr.jump()
            else:
                kb_mgr.set_hold(MoveHold.NEUTRAL)


            # 4. INFO GENERAL + SENS

            st = (
                f"RATON={'ON' if tracker.camera_control_enabled else 'OFF'} "
                f"TECLAS={'ON' if movement_control_enabled else 'OFF'} "
                f"CS2={'SI' if tracker.cs2_active else 'NO'} "
                f"SENS={tracker.sensitivity:.2f}"
            )
            cv2.putText(display, st, (10, h - 55),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.55, (0,255,0), 2)

            cv2.putText(display, "Raton: [C] Teclas [M] Mov [R] Reset Tracker [P] Reset Pose [+/-] Sens",
                        (10, h - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200,200,200), 1)

            cv2.imshow(WIN, display)

            # TECLAS CONTROL PROGRAMA
            key = cv2.waitKey(1) & 0xFF
            if key in (ord('q'), ord('Q'), 27):
                break
            elif key in (ord('c'), ord('C')):
                tracker.camera_control_enabled = not tracker.camera_control_enabled
            elif key in (ord('m'), ord('M')):
                movement_control_enabled = not movement_control_enabled
                if not movement_control_enabled:
                    kb_mgr.set_hold(MoveHold.NEUTRAL)
            elif key in (ord('r'), ord('R')):
                tracker.reset_state()
                kb_mgr.set_hold(MoveHold.NEUTRAL)
            elif key in (ord('p'), ord('P')):
                pose_ctrl.reset()
                movement_control_enabled = False
                kb_mgr.set_hold(MoveHold.NEUTRAL)
                print(">> POSE RESETEADA: vuelve a calibrar (Neutral -> Forward) <<")

            # + / - para sensibilidad del raton 
            elif key in (ord('+'), ord('=')):  # '=' por si el teclado manda '=' al pulsar '+'
                tracker.adjust_mouse_sensitivity(+0.05)
                print(f">> SENS={tracker.sensitivity:.2f} <<")
            elif key in (ord('-'), ord('_')):  # '_' por si viene con shift
                tracker.adjust_mouse_sensitivity(-0.05)
                print(f">> SENS={tracker.sensitivity:.2f} <<")

    finally:
        kb_mgr.release_all()
        pose_ctrl.close()
        cap.release()
        cv2.destroyAllWindows()


main()